# Day 20: Trench Map
https://adventofcode.com/2021/day/20

### Part 1
Start with the original input image and apply the image enhancement algorithm twice, being careful to account for the infinite size of the images. How many pixels are lit in the resulting image?

In [1]:
from typing import Tuple
from advent_of_code import utils

%load_ext blackcellmagic

In [2]:
class Image:
    """Class for containing and enhancing an image."""
    lit_pixels: set
    visible_area_height: int
    visible_area_width: int

    def __init__(self, image_pixels: list) -> None:
        # add only the lit pixels to a set object.
        self.lit_pixels = set(
            (ridx, cidx)
            for ridx, row in enumerate(image_pixels)
            for cidx, pix in enumerate(row)
            if pix == "#"
        )
        # capture the dimensions of the "visible" area to we can print.
        self.visible_area_height = len(image_pixels)
        self.visible_area_width = max([len(r) for r in image_pixels])

    def __repr__(self) -> str:
        """Simple print function for looking that image."""
        image: list = []
        for ridx in range(self.visible_area_height):
            image.append(
                "".join(
                    [
                        "#" if (ridx, cidx) in self.lit_pixels else "."
                        for cidx in range(self.visible_area_width)
                    ]
                )
            )
        return "\n".join(image)

    def enhance(self, algo: list, infinite_bit: int):
        """Runs the image enhacement step, and overwrites the existing image."""

        min_ridx = min(p[0] for p in self.lit_pixels)
        max_ridx = max(p[0] for p in self.lit_pixels)
        min_cidx = min(p[1] for p in self.lit_pixels)
        max_cidx = max(p[1] for p in self.lit_pixels)

        # create a new container since we enhance simultaneously. 
        enhanced_pixels = set()

        # loop through -1 to +1 of the currently lit boundaries.
        for ridx in range(min_ridx - 1, max_ridx + 2):
            for cidx in range(min_cidx - 1, max_cidx + 2):
                index = 0
                
                # loop through the 3x3 square surrounding the pixel
                for sq_r in range(ridx - 1, ridx + 2):
                    for sq_c in range(cidx - 1, cidx + 2):
                        # construct index and if outside of lit area, 
                        # assume infite bit logic.
                        index = index * 10 + (
                            int((sq_r, sq_c) in self.lit_pixels)
                            if (
                                min_ridx <= sq_r <= max_ridx
                                and min_cidx <= sq_c <= max_cidx
                            )
                            else infinite_bit
                        )
                # convert from str rep of binary to dec 
                index = utils.bin_to_dec(str(index))

                # if enhance algo says light then insert into set.
                if algo[index]:
                    enhanced_pixels.add((ridx, cidx))

        self.lit_pixels = enhanced_pixels

    def num_lit_pixels(self) -> int:
        """Returns the number of pixels that are on."""
        return len(self.lit_pixels)


def read_input(input_file: str) -> Tuple[list, Image]:
    with open(input_file) as f:
        image_enhancement: list = [int(x == "#") for x in f.readline().rstrip()]
        image_bits: list = []
        for line in f:
            if line.strip():
                image_bits.append(line.rstrip())

        image = Image(image_bits)

    return image_enhancement, image

In [3]:
img_enhancer, image = read_input(utils.input_location(day=20))
for i in range(2):
    image.enhance(img_enhancer, i & img_enhancer[0])

print(image)
print('----')
print(image.num_lit_pixels())

..#.#...##.#.##.###.#.##.#.##.#####.####.######..#...#.#.#..##.##.##.....##.#..#.####.#.#.##.##.#.##
.....###....##.##...#..#.....######.#.#..#...#..#....#.#.#.#.###.#....#...#...#.##.....#.#.###.#.#.#
###...##.###....#...###.#.#......########....##..#.....#.#...##.###....##..#.#....###...###.##.#....
.#...#..##...######......##.#..#...#####...#.#.#..#.####.##.#.##.......#.#........#.##..#.#.#.#.####
.####.##.#.##.#.#.##.#.##.##.#.#...#####.#.##.##...#.##......####.#..##.#.##.####.....##..###.....#.
.####.#.#..#.....##...##.###.#.#.#..#.#...#..########....###..####...##.#..#.###..###..#.####....#.#
#..#.##.#.#.#....#...##.####..##.#.##..#.###...###..##.###.#.#.##.#.#.#..#####.#...#.##.#..#.##...##
.##..##....#.#..##..#######...#...#....##.#...##...##..#.#.#....#.#.###.##.####.#.##.##.#.#.##.#.###
......#.#....#..######..##......#.####..#.....#.###..#.#....####...#.#.##.####.##.##....###.....####
.#.#.##.##.###.#.#.###...#.....##..###.###.##.#.#..###..##..##..#.##..............#...##.##

Correct answer is 5249

## Part 2
Start again with the original input image and apply the image enhancement algorithm 50 times. How many pixels are lit in the resulting image?

In [4]:
img_enhancer, image = read_input(utils.input_location(day=20))
for i in range(50):
    image.enhance(img_enhancer, i & img_enhancer[0])

print(image)
print('----')
print(image.num_lit_pixels())

##....##.##.##.###.#.#...#.##.....#.......#..........#............####.#..###.#....#####.###.#..#...
#...#.#...#....#####.####...#..#.####..###.##.#######..#.#####.##..###.#.#........######.##########.
..#....#.........#....#.##..#.#.####.##...###..#.##.#...#..##..#...#.#.#.#....##..#.#..###.#.#.#...#
.......##.##....#..##.#....####..#..#..#...#.##..#.##..#.######.######..#..####.##.##..#.#...##.##.#
.#..###.##..####..#.##.#.#..#.....##..#.#..#.##..##.#...#.#.###...##.#...##..#.#.##.....##.#####.#.#
#..#...##....#....#####.#...##...##.#...##...#...#......#.#..#.....####.####.##.##..###.##..#.#.#..#
..#.###.##.##..##....###.###.#.#....##..##....#..#.#..#....##.##...##.#.######.####.....##..#.##..##
.###.#...#..#...##.##..#..##.##.#####.....##.##..##.#.#.....#....##....#.#.#...#######.##..#.##...#.
##.#.###..#....##....#.#....##.#.####.####..###.##.#.#.#.#.##.###....#.#.######.###.#....#...#.#.#.#
.######.##.##.#..#.#..#..#.#.####.#..##....#..#.#...##.....##..##.#..#..#...#####.#..###...

Correct answer is 15714